In [2]:
%load_ext autoreload
%autoreload 2

import yeahml as yml

## Set up

First, we need to make sure a configuration file has been created and follows the reccomended format

In [3]:
!cat ./examples/mnist/main_config.yaml

meta:
  name: 'mnist'
  experiment_dir: 'trial_01'
  # TODO: information on when to save params, currently only best params saved
logging:
  console:
    level: 'info'
    format_str: null
  file:
    level: 'ERROR'
    format_str: null
  graph_spec: True

performance:
  loss_fn: 
    type: 'categorical_crossentropy'
  type: ["MeanSquaredError", "TopKCategoricalAccuracy", "CategoricalAccuracy"]
  options: [null, 
            {"k": 2}, 
            null]

# TODO: this section needs to be redone
data:
  in:
    dim: [784]
    dtype: 'float32'
    reshape_to: [28, 28, 1]
  label:
    dim: [10]
    dtype: 'float32'
    one_hot: True # TODO: ensure this is being used
  TFR_parse:
    feature:
      name: "/image"
      dtype: "int8"
      tftype: "fixedlenfeature"
      in_type: "string"
    label:
      name: "/label"
      dtype: "int64" # what to decode it to
      tftype: "fixedlenfeature"
      in_type: "int64" # what it is encoded as
  TFR:
    dir: './examples/mnist/data/'
    train:

## Parse configuration files

Next, we will parse out the relevant bits of information into modular components.

In [7]:
## Core Logic
example = "./examples/mnist/main_config.yaml"
config_dict = yml.create_configs(example)

-8090147948956741930


## Examine configs

Each of these configs is a simple python dictionary with the expected format for yeahml.

They can be extracted/created seperately if desired (an example of modifying the `model_cdict` is shown next)

In [8]:
config_dict

{'meta': {'name': 'mnist',
  'experiment_dir': 'trial_01',
  'log_dir': './examples/mnist/trial_01'},
 'logging': {'log_c_lvl': 'INFO',
  'log_c_str': '%(name)-12s: %(levelname)-8s %(message)s',
  'log_f_lvl': 'ERROR',
  'log_f_str': '[%(asctime)s - %(filename)s:%(lineno)s - %(funcName)20s()][%(levelname)-8s]: %(message)s',
  'log_g_lvl': 'DEBUG',
  'log_g_str': '%(name)-12s: %(levelname)-8s %(message)s',
  'log_p_lvl': 'DEBUG',
  'log_p_str': '[%(levelname)-8s] %(message)s'},
 'performance': {'loss_fn': {'type': 'categorical_crossentropy'},
  'met_list': ['meansquarederror',
   'topkcategoricalaccuracy',
   'categoricalaccuracy'],
  'met_opts_list': [None, {'k': 2}, None]},
 'data': {'input_layer_dim': [-1, 28, 28, 1],
  'in_dim': [None, 784],
  'in_dtype': 'float32',
  'reshape_in_to': [-1, 28, 28, 1],
  'output_dim': [None, 10],
  'label_dtype': 'float32',
  'label_one_hot': True,
  'TFR_dir': './examples/mnist/data/',
  'TFR_train': 'train.tfrecords',
  'TFR_test': 'test.tfrecords'

## Build the model
(according to our specifications)

In [34]:
model = yml.build_model(config_dict)

build_logger: INFO     -> START building graph
build_logger: INFO     -> START building hidden block
graph_logger: INFO     | conv_1          | no_shape
graph_logger: INFO     | conv_2          | no_shape
graph_logger: INFO     | dropout_1       | no_shape
graph_logger: INFO     | pool_1          | no_shape
graph_logger: INFO     | flatten_1       | no_shape
graph_logger: INFO     | dense_1         | no_shape
graph_logger: INFO     | dense_2_output  | no_shape
build_logger: INFO     [END] building hidden block
build_logger: INFO     information json file created


In [10]:
# this is a standard tf2 model
print(model.summary())

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv_1 (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
conv_2 (Conv2D)              (None, 24, 24, 64)        18496     
_________________________________________________________________
dropout_1 (Dropout)          (None, 24, 24, 64)        0         
_________________________________________________________________
pool_1 (AveragePooling2D)    (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                294944

## Train the model

In [11]:
train_dict = yml.train_model(model, config_dict)

train_logger: INFO     -> START training graph
train_logger: INFO     start creating train_dict
train_logger: INFO     [END] creating train_dict


In [12]:
# the returned train_dict contains relevant information from training
print(train_dict)

{'train_losses': [5.2153835, 0.9760489, 0.8170814], 'val_losses': [0.7062942, 0.56644136, 0.5246587], 'epochs': [0, 1, 2], 'meansquarederror': 0.02217643, 'val_meansquarederror': 0.024458267, 'topkcategoricalaccuracy': 0.93845457, 'val_topkcategoricalaccuracy': 0.92425454, 'categoricalaccuracy': 0.8475091, 'val_categoricalaccuracy': 0.8289818}


## Let's try a different model

We can create a new model by simply specifying the model archicture "on the fly"

In [13]:
new_model = '''
meta:
  name: "model_b"
  name_override: True
  activation:
    type: 'elu'

layers:
  conv_1:
    type: 'conv2d'
    options:
      filters: 32
      kernel_size: 3
  conv_2:
    type: 'conv2d'
    options:
      filters: 64
      kernel_size: 3
  dropout_1:
    type: 'dropout'
    options:
      rate: 0.5
  pool_1:
    type: 'AveragePooling2D'
    options:
      pool_size: 2
      strides: 2
  flatten_1:
    type: 'flatten'
  dense_1:
    type: 'dense'
    options:
      units: 16
      kernel_regularizer:
        type: 'L1'
        l: 0.01
  dense_2_output:
    type: 'dense'
    options:
      units: 10
      activation:
        type: 'softmax'
'''

# use the specified configuration method to ensure proper formatting
new_model_conf = yml.config.config_model(new_model, config_dict)

8226787121120433438


## Build and Train

In [14]:
model_b = yml.build_model(new_model_conf)
train_dict = yml.train_model(model_b, new_model_conf)

build_logger: INFO     -> START building graph
build_logger: INFO     -> START building hidden block
graph_logger: INFO     | conv_1          | no_shape
graph_logger: INFO     | conv_2          | no_shape
graph_logger: INFO     | dropout_1       | no_shape
graph_logger: INFO     | pool_1          | no_shape
graph_logger: INFO     | flatten_1       | no_shape
graph_logger: INFO     | dense_1         | no_shape
graph_logger: INFO     | dense_2_output  | no_shape
build_logger: INFO     [END] building hidden block
train_logger: INFO     -> START training graph
train_logger: INFO     start creating train_dict
train_logger: INFO     [END] creating train_dict


In [15]:
print(train_dict)

{'train_losses': [4.971495, 1.0700443, 0.7730109], 'val_losses': [0.6853227, 0.4991753, 0.35921574], 'epochs': [0, 1, 2], 'meansquarederror': 0.018187756, 'val_meansquarederror': 0.017009854, 'topkcategoricalaccuracy': 0.95281816, 'val_topkcategoricalaccuracy': 0.95887274, 'categoricalaccuracy': 0.8790182, 'val_categoricalaccuracy': 0.88336366}


## Evaluate Model

In [16]:
eval_dict = yml.eval_model(
    model_b,
    new_model_conf
)
print(eval_dict)

eval_logger : INFO     params loaded from examples/mnist/trial_01/model_b/save/params/run_2019_12_22-12_51_19/best_params.h5
eval_logger : INFO     -> START evaluating model
eval_logger : INFO     [END] evaluating model
eval_logger : INFO     -> START creating eval_dict
eval_logger : INFO     [END] creating eval_dict


{'meansquarederror': 0.017009858, 'topkcategoricalaccuracy': 0.95887274, 'categoricalaccuracy': 0.88336366}


In [1]:
eval_dict = yml.eval_model(
    model,
    config_dict
)
print(eval_dict)

NameError: name 'yml' is not defined